In [89]:
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta

In [90]:
import requests
from bs4 import BeautifulSoup
import html2text
from datetime import datetime

In [91]:
def get_minutes_list(from_date='20050101'):
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')

    for pageIndex in range(1, 31):
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers ={'User-Agent' : user_agent}
        page = requests.get(url, headers=headers)

        try:
            soup = BeautifulSoup(page.content, 'html.parser')
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
            datainfo = soup.find_all('div', class_='col s12 dataInfo')

            for post, data in zip(brdList, datainfo):
                guid = prefix_addr+post.a['href'][:-12]

                desPage = requests.get(guid)
                desSoup = BeautifulSoup(desPage.content, 'html.parser')   
                description = desSoup.find('div', class_='dbData').get_text().strip()
                if description.replace(' ','').find('통화정책방향') >= 0:
                    title = post.find('span', class_='titlesub').get_text().strip()
        
                    mdate = title[title.find(')(') + 2:-1]            
                    if mdate[-1] == '.':
                        mdate = mdate[:-1]
                    mdate = datetime.strptime(mdate, '%Y.%m.%d')

                    if mdate < from_date:
                        break

                    rdate = data.find('span', class_='date').get_text().strip()
                    rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')    

                    get_minutes_file(guid, mdate, rdate)
        except:
            print("get url.content error and pass page{} it".format(pageIndex))

In [92]:
def get_minutes_file(page_addr, mdate, rdate):
    file_header = 'data/minutes/pdf/'
    prefix_addr = "http://bok.or.kr"

    page = requests.get(page_addr)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        links = soup.find('div', class_='addfile').find_all('a')

        for link in links:
            filename = link.get_text()
            filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')

            if filename[-3:] == 'pdf':
                filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d')+ '.pdf'
                file_addr = prefix_addr + link["href"]
                file_res = requests.get(file_addr)
                filepath = file_header + filename

                with open(filepath, 'wb') as f:
                    f.write(file_res.content)

                print('save file name : ' + filename)
    except:
        print("get file failed and pass it")

In [93]:
get_minutes_list()

save file name : 20190418_20190507.pdf
save file name : 20190228_20190319.pdf
save file name : 20190124_20190212.pdf
save file name : 20181130_20181218.pdf
save file name : 20181018_20181106.pdf
save file name : 20180831_20180918.pdf
save file name : 20180712_20180731.pdf
save file name : 20180524_20180612.pdf
save file name : 20180412_20180502.pdf
save file name : 20180227_20180320.pdf
save file name : 20180118_20180206.pdf
save file name : 20171130_20171219.pdf
save file name : 20170831_20170919.pdf
save file name : 20170713_20170801.pdf
save file name : 20170525_20170613.pdf
save file name : 20170413_20170502.pdf
save file name : 20170223_20170314.pdf
save file name : 20170113_20170131.pdf
save file name : 20161215_20170103.pdf
save file name : 20161111_20161129.pdf
save file name : 20161013_20161101.pdf
save file name : 20160909_20160927.pdf
save file name : 20160811_20160830.pdf
save file name : 20160714_20160802.pdf
save file name : 20160609_20160628.pdf
save file name : 20160513

In [72]:
import re
from tika import parser
from os import listdir
from os.path import isfile, join

# 폴더내 pdf파일을 txt 파일로 변경
def pdf2txt (source_folder="./data/minutes/pdf/", output_folder="./data/minutes/text/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        # pdf 파일을 text로 변환
        print(pdf_filepath)
        parsedPDF = parser.from_file(pdf_filepath)["content"]
        
        with open((output_folder + pdf).replace('.pdf', '.txt'), 'w') as f :
            f.write(parsedPDF)
            f.close()    
            
pdf2txt()

./data/minutes/pdf/20130214_20130305.pdf
./data/minutes/pdf/20081107_20081223.pdf
./data/minutes/pdf/20151015_20151103.pdf
./data/minutes/pdf/20080612_20080729.pdf
./data/minutes/pdf/20160419_20160504.pdf
./data/minutes/pdf/20070111_20070227.pdf
./data/minutes/pdf/20180227_20180320.pdf
./data/minutes/pdf/20070712_20070828.pdf
./data/minutes/pdf/20140313_20140401.pdf
./data/minutes/pdf/20121109_20121127.pdf
./data/minutes/pdf/20070510_20070626.pdf
./data/minutes/pdf/20050609_20050726.pdf
./data/minutes/pdf/20160609_20160628.pdf
./data/minutes/pdf/20121011_20121030.pdf
./data/minutes/pdf/20180118_20180206.pdf
./data/minutes/pdf/20130314_20130402.pdf
./data/minutes/pdf/20141211_20141230.pdf
./data/minutes/pdf/20150217_20150310.pdf
./data/minutes/pdf/20110310_20110426.pdf
./data/minutes/pdf/20090212_20090331.pdf
./data/minutes/pdf/20090512_20090623.pdf
./data/minutes/pdf/20130808_20130827.pdf
./data/minutes/pdf/20071011_20071127.pdf
./data/minutes/pdf/20170831_20170919.pdf
./data/minutes/p

In [73]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [79]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [90]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens+ ngrams

In [ ]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join

    
def preprocessing(source_folder="./data/minutes/text/", output_file="./data/minutes/minutes_preprocessed.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for txt_file in txt_files :
        with open(source_folder + txt_file, 'r') as f :
            txt = f.read()
            sections, section_texts = preprocess_minutes(txt)
            
            print(txt_file)
            
            ngrams = []
            for section_text in section_texts[1:3] :
                for sentence in section_text :
                    ngrams.append(','.join(text2ngram(sentence)))
                
            #print('@@@'.join(ngrams))
            #break

            df.loc[len(df)] = [txt_file.split('_')[0],
                               txt,
                               '@@@'.join(section_texts[0]),
                               '@@@'.join(section_texts[1]),
                               '@@@'.join(section_texts[2]),
                               '@@@'.join(section_texts[3]),
                               '@@@'.join(section_texts[4]),
                               '@@@'.join(section_texts[5]),
                               len(section_texts[0]),
                               len(section_texts[1]),
                               len(section_texts[2]),
                               len(section_texts[3]),
                               len(section_texts[4]),
                               len(section_texts[5]),
                               '@@@'.join(ngrams)]
    
    df.to_csv(output_file, index=False)
        
    print(df.shape)
preprocessing()

20050609_20050726.txt
20050707_20050823.txt
20050811_20050927.txt
20050908_20051025.txt
20051011_20051129.txt
20051110_20051227.txt
20051208_20060124.txt
20060112_20060228.txt
20060209_20060328.txt
20060309_20060425.txt
20060407_20060523.txt
20060511_20060627.txt
20060608_20060725.txt
20060707_20060822.txt
20060810_20060926.txt
20060907_20061024.txt
20061012_20061128.txt
20061109_20061226.txt
20061207_20070123.txt
20070111_20070227.txt
20070208_20070327.txt
20070308_20070424.txt


In [5]:
import pandas as pd
df = pd.read_csv("./data/minutes/minutes_preprocessed.csv")
df['ngram'][0]

'경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없/VA,불구/NNG,원엔/NNG,환율/NNG,지속/NNG,로/NNG,하락/NNG,하/XSV,이유/NNG,물/VV,대해/VV,위안/NNG,절/NNG,상시/NNG,권/NNG,통화/NNG,절상/NNG,크/VA,기대/NNG,원화/NNG,달러/NNG,엔달러/NNG,환율/NNG,상승/NNG,추수/NNG,하/XSV,못하/VX,외환/NNG,수급/NNG,면/NNG,공급우위/NNG,상태/NNG,지속/NNG,되/XSV,국제투자은행/NNG,엔달러/NNG,환율/NNG,전망/NNG,상향/NNG,하/XSV,불구/NNG,원달러/NNG,조정/NNG,아직/MAG,이루/VV,않/VX,기업/NNG,기대/NNG,원화/NNG,절상/NNG,편향/NNG,되/XSV,기인/NNG,하/VV,설명/NNG,하/XSV,엔달러/NNG;환율/NNG;상승/NNG,기대/NNG;원화/NNG;절상/NNG,통화/NNG;절상/NNG,전망/NNG;상향/NNG@@@환율/NNG,장기/NNG,각국/NNG,기초/NNG,경제/NNG,여건/NNG,반영/NNG,하/XSV,움직이/VV,점/NNG,감안/NNG,하/VV,때/NNG,원엔/NNG,환율/NNG,하락/NNG,일시적/VAX,현상/NNG,장기/NNG,반전/NNG,되/VV,가능성/NNG,있/VA,보인다는/VV,견해/NNG,밝혔으며/VV,대해/VV,견해/NNG,동의/NNG,하/XSV,시장/NNG,규모/NNG,작/VA,일시적/VAX,외부충격/NNG,의하/VV,영향/NNG,쉽/VA,받/VV,점/NNG,설명/NNG,하/XSV,내외/NNG,금리/NNG,역전/NNG,나타나/VV,같/VA,대규모/NNG,외환/NNG,유입/NNG,일어나/VV,않/VX,감안/NNG,하/VV,때/NNG,원엔/NNG,환율/NNG,추가/NNG,하락/NNG,하/VV,점차/MAG,축소/NNG,되/VV,생각/NNG,되/VV,답변/NNG,하/XSV,원엔/NNG;환율/NNG;하락/NNG,일시적/VAX;외부충격/NNG,하락/NNG;일시적/VAX@@@유로존/N

In [4]:
df

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,20050609,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 소비재판매액 증가세가 다소 둔화되고 있으나 GDP 상의 소비와...,일부 위원은 우리나라 경제가 일본경제 에 비해 더 나아질 특별한 요인이 없음에도 ...,일부 위원은 M1이 실물경제와 어느 정도의 시차를 두고 밀접한 연관성을 지니는지...,NaN,NaN,NaN,19,4,160,0,0,0,"경제/NNG,경제/NNG,비해/VV,더/MAG,낫/VV,특별/NNG,요인/NNG,없..."
1,20050707,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 최근 이란의 새 대통령 취임.@@@이후 자국경제 보호주의 강화 등으로 ...,일부 위원은 6월 들어 국제유가가 크게 오르고 원화와 엔화간 동조화 현상이 뚜렷...,일부 위원은 단기적으로 대출이 급격히 증가 한 후에는 반드시 조정이 이어지고 민간...,NaN,NaN,NaN,13,5,160,0,0,0,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/..."
2,20050811,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 소비, 수출, 건설투자가 견조한 증가세를 보이고 물가도 안정세를 이...",일부 위원은 위안화 절상 이후 아시아 통화중 우리나라 원화의 절상폭이 가장 컸다...,일부 위원은 최근 장기시장금리 상승에도 불 구하고 기업 자금조달은 오히려 확대되고...,NaN,NaN,NaN,11,12,159,0,0,0,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG..."
3,20050908,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 7월 전망시보다 높은 수준을 지속하고 있는 국제유가가 내년도 경제성...,일부 위원은 최근 인도네시아 금융불안 사태가 발생한 가운데 국제 환투기세력이 인...,일부 위원은 시장이 제대로 작동한다면 비우 - 5 - 량등급의 회사채 가산금...,NaN,NaN,NaN,6,4,159,0,0,0,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기..."
4,20051011,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 지방자치단체의 투자유치활동 이 활발해지고 항공기, 철도, 자동차 등의...",일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,NaN,NaN,NaN,11,4,158,0,0,0,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
5,20051110,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 향후 미국의 안정실업률 (NAIRU: Non Accelerating ...,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",일부 위원은 장단기금리 격차가 경제성장률에 선행하는 모습을 보인다고 하면 미국의...,NaN,NaN,NaN,14,8,157,0,0,0,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V..."
6,20051208,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 경제성장률 전망치가 전년동기 대비로 보면 내년 하반기에 둔화되는 것으...,일부 위원은 금년 들어 달러화가 엔화 나 유로화에 대해 상당히 강세를 나타낸 것과...,일부 위원은 우리나라와 미국의 장기시장금리 를 비교해볼 때 최근 우리나라의 장기시...,NaN,NaN,NaN,13,3,179,0,0,0,"들/VV,달러/NNG,엔/NNG,유로/NNG,대해/VV,상당히/MAG,강세/NNG,..."
7,20060112,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,일부 위원은 2002년부터 거의 예외 없 이 연초에는 원화의 강세 현상이 나타나 ...,일부 위원은 실물활동 중 민간소비나 GDP 증가율이 M1증가율과 밀접한 관련성이...,NaN,NaN,NaN,0,12,162,0,0,0,"거의/MAG,예외/NNG,없/VA,원화/NNG,강세/NNG,현상/NNG,나타나/VV..."
8,20060209,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 우리나라의 대 중국 수출품목 중 원자재나 중간재가 차지하는 비중이 ...,일부 위원은 자본유입에 대해서는 관대 한 반면 자본유출에 대해서는 비교적 엄격한 ...,일부 위원은 최근 주가가 크게 변동한 이유 중 하나로 기관투자가들이 단기적 관점...,NaN,NaN,NaN,6,4,156,0,0,0,"자본/NNG,유입/NNG,대해서/VV,관대/NNG,반면/NNG,자본/NNG,유출/N..."
9,20060309,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 최근 다소의 긴축으로 선회한 미국, 유럽, 일본 등의 통화정책 기조...",일부 위원은 과거 사례를 보면 원화환 율은 엔화환율 및 위안화환율과 높은 상관관계...,일부 위원은 지난 해 10월 이후 세 차례에 걸 친 콜금리 목표 인상에도 불구하고...,NaN,NaN,NaN,5,5,156,0,0,0,"사례/NNG,보/VV,원화/NNG,환율/NNG,엔/NNG,환율/NNG,위안/NNG,..."


In [4]:
df[['date','Economic Situation count', 'Foreign Currency count','Financial Markets count','Monetary Policy count','Participant Views count','Government View count']].to_csv("./results/minutes_daily_count.csv", index = False)